### Modifications effectuées manuellement :
- Suppression des deux premières lignes de chaque fichier (ligne vide et ligne titre)
- Suppression des messages qui sont des traductions de messages en français
- Division du corpus selon les 3 moments de la dissolution

## Objectifs de ce notebook
- Mettre les différents fichiers séparés dans un dataframe
- Lemmatiser le corpus
- Nettoyer le corpus des caractères spéciaux, majuscules, sauts de lignes, stopwords etc.

In [43]:
# On rassemble les textes dans un dictionnaire

temp = ['posts_gi1','posts_gi2','posts_TE1', 'posts_TE2', 'posts_TE3']
temp_full = ['GI_pré-dissolution','GI_dissolution','TE_pré-dissolution', 'TE_dissolution', 'TE_post-dissolution']
data = {}
for i, m in enumerate(temp):
    with open ('corpus_temp/' + m + ".txt", encoding="utf8") as f:
        data[temp_full[i]] =  f.read()
data

{'GI_pré-dissolution': 'Bonne année à tous ! Que 2021 soit une excellente année pour tous les défenseurs de l\'identité. A l\'abordage !\n📢 Le vrai visage de ceux qui demandent la dissolution de Génération Identitaire et la fermeture de la Traboule, votre maison de l’identité ⬇️\nEn 2012, Génération Identitaire s\'était fait connaître grâce à cette vidéo ⬆️ intitulée "La Déclaration de Guerre". Connaissant un très vif succès en France, elle avait ensuite été spontanément traduite dans différents pays européens. Cet élan avait participé ainsi au développement du mouvement identitaire sur tout notre continent.\nGavées d’argent public, les Minguettes restent un bastion de l’islam radical La Cour des Comptes dénonce dans un récent rapport le séparatisme en cours dans le quartier des Minguettes à Vénissieux, et décrit ce quartier comme « l’un des bastions de l’islam fondamentaliste\xa0» en France. Malgré la fermeture de deux mosquées salafistes suite aux attentats de Paris en 2015, ce quart

In [44]:
# On importe le résultat dans un dataframe

import pandas as pd

pd.set_option('max_colwidth',120)

data_df = pd.DataFrame(data, index = [0]).transpose()
data_df.columns = ['messages']
data_df

,messages
GI_pré-dissolution,Bonne année à tous ! Que 2021 soit une excellente année pour tous les défenseurs de l'identité. A l'abordage !\n📢 Le...
GI_dissolution,Darmanin est scandalisé... que de jeunes Français aient réclamé l'application de la loi ? L'entrée de migrants cland...
TE_pré-dissolution,Merci à vous tous qui me rejoignez sur Telegram. Le monopole des GAFA sur internet n’a que trop duré.Le mandat de Bi...
TE_dissolution,Vous souhaitez nous aider ?Alors c'est le moment d'agir.Darmanin a annoncé vouloir dissoudre Génération Identitaire....
TE_post-dissolution,🗣 Photos prises ce matin alors que les forces de l’ordre nous bloquaient l’accès au pont où nous souhaitions faire n...


## Lemmatisation

In [45]:
import spacy
nlp = spacy.load('fr_core_news_md')

data_lem = data_df.copy(deep = True)
data_lem['messages'] = data_df['messages'].apply(lambda x: " ".join([y.lemma_ for y in nlp(x)]))
data_lem

,messages
GI_pré-dissolution,bon année à tout ! que 2021 être un excellent année pour tout le défenseur de le identité . avoir le abordage ! \n 📢...
GI_dissolution,Darmanin être scandaliser ... que de jeune français avoir réclamer le application de le loi ? le entrée de migrant c...
TE_pré-dissolution,merci à vous tout qui me rejoindre sur Telegram . le monopole de GAFA sur internet n’ avoir que trop durer . le mand...
TE_dissolution,vous souhaiter nous aider ? alors ce être le moment de agir . Darmanin avoir annoncer vouloir dissoudre génération I...
TE_post-dissolution,🗣 photo prendre ce matin alors que le force de l’ ordre nous bloquer l’ accès au pont où nous souhaiter faire notre ...


## Supression du brouhaha

In [46]:
import re
import string

def clean_text(text):
    text = text.lower()
    text = re.sub('[?!/,()«»:;]', '', text)
    text = re.sub('[/.\'’"]', ' ', text)
    text = re.sub('\n', ' ', text)
    return text

In [47]:
data_clean = data_lem.copy(deep = True)
data_clean = pd.DataFrame(data_lem.messages.apply(lambda x: clean_text(x)))
data_clean

,messages
GI_pré-dissolution,bon année à tout que 2021 être un excellent année pour tout le défenseur de le identité avoir le abordage 📢 le...
GI_dissolution,darmanin être scandaliser que de jeune français avoir réclamer le application de le loi le entrée de migrant cl...
TE_pré-dissolution,merci à vous tout qui me rejoindre sur telegram le monopole de gafa sur internet n avoir que trop durer le mand...
TE_dissolution,vous souhaiter nous aider alors ce être le moment de agir darmanin avoir annoncer vouloir dissoudre génération id...
TE_post-dissolution,🗣 photo prendre ce matin alors que le force de l ordre nous bloquer l accès au pont où nous souhaiter faire notre ...


## Stopwords

In [48]:
from nltk.corpus import stopwords 

stop_words = set(stopwords.words('french')) 

data_clean2 = data_clean.copy(deep = True)
data_clean2['messages'] = data_clean['messages'].str.split()
data_clean2['messages'] = (data_clean2['messages'].apply(lambda x: [w for w in x if w not in stop_words]))
data_clean2['messages'] = [' '.join(map(str, l)) for l in data_clean2['messages']]
data_clean2

,messages
GI_pré-dissolution,bon année tout 2021 être excellent année tout défenseur identité avoir abordage 📢 vrai visage celui demander dissolu...
GI_dissolution,darmanin être scandaliser jeune français avoir réclamer application loi entrée migrant clandestin france être illéga...
TE_pré-dissolution,merci tout rejoindre telegram monopole gafa internet avoir trop durer mandat biden commencer bien targue être démocr...
TE_dissolution,souhaiter aider alors être moment agir darmanin avoir annoncer vouloir dissoudre génération identitaire avoir besoin...
TE_post-dissolution,🗣 photo prendre matin alors force ordre bloquer accès pont où souhaiter faire action contester dissolution acter mat...


In [49]:
import pickle

data_clean2.to_pickle('corpus.pkl')

## CountVectorizer

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
data_cv = cv.fit_transform(data_clean2.messages)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean2.index
data_dtm

,000,10,100,1000,1007,10h,10h15,10h30,12,12h25,...,étienne,étranger,étudiant,étudier,éventuel,évidemment,éviter,évoque,événement,être
GI_pré-dissolution,4,0,1,0,1,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,37
GI_dissolution,4,1,0,0,0,0,0,0,2,0,...,0,0,2,1,0,0,0,1,1,72
TE_pré-dissolution,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,27
TE_dissolution,3,3,0,0,0,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,43
TE_post-dissolution,0,0,0,0,0,0,0,0,0,0,...,0,2,2,0,0,1,1,0,2,61


In [51]:
data_dtm.to_pickle("dtm.pkl")

## Pour les Word Embeddings
- On effectue les mêmes modifications, mais en gardant l'indicateur de saut de ligne pour pouvoir ensuite bien séparer les différents messages

In [52]:
def clean_text_WE(text):
    text = text.lower()
    text = re.sub('[?!/,()«»:;“”]', '', text)
    text = re.sub('[/.\'’"]', ' ', text)
    return text

In [55]:
data_clean_WE = pd.DataFrame(data_lem.messages.apply(lambda x: clean_text_WE(x)))
data_clean_WE

,messages
GI_pré-dissolution,bon année à tout que 2021 être un excellent année pour tout le défenseur de le identité avoir le abordage \n 📢 l...
GI_dissolution,darmanin être scandaliser que de jeune français avoir réclamer le application de le loi le entrée de migrant cl...
TE_pré-dissolution,merci à vous tout qui me rejoindre sur telegram le monopole de gafa sur internet n avoir que trop durer le mand...
TE_dissolution,vous souhaiter nous aider alors ce être le moment de agir darmanin avoir annoncer vouloir dissoudre génération id...
TE_post-dissolution,🗣 photo prendre ce matin alors que le force de l ordre nous bloquer l accès au pont où nous souhaiter faire notre ...


In [56]:
data_clean_WE.to_pickle('corpus_WE.pkl')